#### Import Dependencies

In [2]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import toml
import pandas as pd
from prophet import Prophet
import json

#### Load the credentials from toml and connect to Snowflake

In [3]:
secrets = toml.load("secrets.toml")

accountname = secrets["SNOWFLAKE"]["account"]
user = secrets["SNOWFLAKE"]["user"]
password = secrets["SNOWFLAKE"]["password"]
role = secrets["SNOWFLAKE"]["role"]
database = secrets["SNOWFLAKE"]["database"]
schema = secrets["SNOWFLAKE"]["schema"]
warehouse = secrets["SNOWFLAKE"]["warehouse"]

connection_parameters = {
    "account": accountname,
    "user": user,
    "password": password,
    "role": role,
    "database": database,
    "schema": schema,
    "warehouse": warehouse,
    "ocsp_fail_open":"False"
}

session = Session.builder.configs(connection_parameters).create()

In [4]:
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

[Row(CURRENT_WAREHOUSE()='STOCK_PRED_WH', CURRENT_DATABASE()='STOCK_PRED_DB', CURRENT_SCHEMA()='STOCK_PRED_SCHEMA')]


#### Create a stage to store the procedure in snowflake

In [5]:
session.sql('CREATE OR REPLACE STAGE ML_MODELS').collect()

[Row(status='Stage area ML_MODELS successfully created.')]

In [6]:
pd.DataFrame(session.sql('SHOW STAGES').collect())

,created_on,name,database_name,schema_name,url,has_credentials,has_encryption_key,owner,comment,region,type,cloud,notification_channel,storage_integration
0,2023-06-20 00:52:23.656000-07:00,ML_MODELS,STOCK_PRED_DB,STOCK_PRED_SCHEMA,,N,N,ACCOUNTADMIN,,None,INTERNAL,None,None,None
1,2023-06-19 00:54:41.392000-07:00,PRICE_PRED_GCS_STAGE,STOCK_PRED_DB,STOCK_PRED_SCHEMA,gcs://data_lake_datatalks-386314/,N,N,ACCOUNTADMIN,,,EXTERNAL,GCP,None,GCS_INTEGRATION


### Create a procedure

In [14]:
def sproc_predict_using_prophet(session: Session, 
                                training_table: str,
                                include_history: str,
                                period: int) -> T.Variant:
    
    # Loading data into pandas dataframe
    data_sdf = session.table(training_table)    
    data = data_sdf.select('DATE','CLOSE').collect()
    data = pd.DataFrame(data, columns=['ds', 'y'])
    data.dropna(axis=0, inplace=True) 
    data.drop_duplicates(subset='ds', keep="last", inplace=True)
    data.sort_values(by='ds', inplace=True)
    
    # Actual model training
    from prophet import Prophet
     
    model = Prophet(changepoint_prior_scale=0.5,
                    seasonality_prior_scale=0.01,
                    seasonality_mode='additive',
                    changepoint_range=0.99
                       )
    model.fit(data)
    
    if include_history == 'Y':
        flag = True
    else:
        flag = False
        
    future_df = model.make_future_dataframe(periods=period,
                                           include_history=flag)
    forecast = model.predict(future_df)
  
    return forecast.to_dict()

In [15]:
session.add_packages('snowflake-snowpark-python','prophet')
# Create stored procedure
sproc_predict_using_prophet = session.sproc.register(func=sproc_predict_using_prophet, 
                                            name='sproc_predict_using_prophet', 
                                            is_permanent=True, 
                                            replace=True, 
                                            stage_location='@ML_MODELS', 
                                            packages=['snowflake-snowpark-python','prophet'])

### Test the procedure

In [16]:
# run the stored procedure for 10 futur days
training_table = 'historical_prices'
show_history='N'
future_days=10

pred_list = session.sql(
            "call sproc_predict_using_prophet('{}', '{}',{})".format(training_table,show_history, future_days)   
            ).collect()

pred_df = pd.DataFrame(json.loads(pred_list[0][0]))
pred_df = pred_df[['ds','yhat']]
pred_df['ds'] = pd.to_datetime(pred_df['ds']).dt.date
pred_df.columns = ['DATE', 'PRICE']
pred_df

,DATE,PRICE
0,2023-06-20,162.082714
1,2023-06-21,162.144327
2,2023-06-22,162.151188
3,2023-06-23,162.189951
